# **Cluster Configuration**

1. **Cluster Resources:**
- **Master:** n2-standard-4 (4 vCPUs, 16 GB RAM, 32GB disk)
- **Workers (2x):** n2-standard-4 (4 vCPUs, 16 GB RAM, 64GB disk each)
- **Total:** 8 worker vCPUs, ~32 GB RAM (excluding master node)
2. **Dataproc Features Disabled**:
- No **autoscaling, Metastore, advanced execution layer, advanced optimizations**
- **Storage**: pd-balanced (no SSDs, so I/O optimization is crucial)
- **Networking**: Internal IP **enabled**
3. Optimization Strategy:
- Tune **shuffle partitions, broadcast join threshold, and storage persistence**
- Adjust **parallelism** based on **2 workers x 4 cores**
- Avoid **excessive caching** due to **disk-based storage**

In [2]:
from pyspark.sql import SparkSession

## **Some partitions configs and their usages**
- spark.executor.memory: using 6GB out of 16 keeping some for sys operations
- spark.executor.cores: using max cores of our setup (i.e. 4)
- spark.executor.instances: using max instance of our setup (i.e. number of worker nodes)
- spark.driver.memory: driver memory
- spark.driver.maxResultSize: max size of a partition
- spark.sql.shuffle.partitions: no of partitions during joins/aggregations
- spark.default.parallelism: no of paritions in RDD returned by transformations
- spark.sql.adaptive.enabled: reoptimzie queries during execution
- spark.sql.adaptive.coalescePartition.enabled: coalesces continuous paritions
- spark.sql.autoBroadcastJoinThreshold: max size send over a broadcast
- spark.sql.files.maxPartitionBytes: max size in a partition when reading
- spark.sql.files.openCostInBytes: cost to open a file (reads that many bytes)
- spark.memory.fraction: fraction in memory
- spark.memory.storageFraction: fraction in storage

In [4]:
spark = SparkSession.builder\
.appName('Olist Ecommerce Optimized')\
.config('spark.executor.memory', '6g')\
.config('spark.executor.cores', '4')\
.config('spark.executor.instances', '2')\
.config('spark.driver.memory', '4g')\
.config('spark.driver.maxResultSize', '2g')\
.config('spark.sql.shuffle.partitions', '64')\
.config('spark.default.parallelism', '64')\
.config('spark.sql.adaptive.enabled', 'true')\
.config('spark.sql.adaptive.coalescePartition.enabled', 'true')\
.config('spark.sql.autoBroadcastJoinThreshold', 20*1024*1024)\
.config('spark.sql.files.maxPartitionBytes', '64MB')\
.config('spark.sql.files.openCostInBytes', '2MB')\
.config('spark.memory.fraction', '0.8')\
.config('spark.memory.storageFraction', '0.2')\
.getOrCreate()

25/05/25 23:03:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
hdfs_path = '/data/olist/'

In [6]:
customers_df = spark.read.csv(hdfs_path + 'olist_customers_dataset.csv', inferSchema='true', header='true')
geolocation_df = spark.read.csv(hdfs_path + 'olist_geolocation_dataset.csv', inferSchema='true', header='true')
order_items_df = spark.read.csv(hdfs_path + 'olist_order_items_dataset.csv', inferSchema='true', header='true')
order_payments_df = spark.read.csv(hdfs_path + 'olist_order_payments_dataset.csv', inferSchema='true', header='true')
order_reviews_df = spark.read.csv(hdfs_path + 'olist_order_reviews_dataset.csv', inferSchema='true', header='true')
orders_df = spark.read.csv(hdfs_path + 'olist_orders_dataset.csv', inferSchema='true', header='true')
products_df = spark.read.csv(hdfs_path + 'olist_products_dataset.csv', inferSchema='true', header='true')
sellers_df = spark.read.csv(hdfs_path + 'olist_sellers_dataset.csv', inferSchema='true', header='true')
product_category_translation_df = spark.read.csv(hdfs_path + 'product_category_name_translation.csv', inferSchema='true', header='true')

In [7]:
complete_df = spark.read.parquet('/data/olist/olist-merged')

In [ ]:
complete_df.printSchema()

## Optimized Joins

### **Broadcast Join**

In [10]:
from pyspark.sql.functions import *
customers_broadcast_df = broadcast(customers_df)
optimized_broadcast_df = complete_df.join(customers_broadcast_df, 'customer_id')

### **Sort & Merge Join**

In [12]:
sorted_customers_df = customers_df.sortWithinPartitions('customer_id')
sorted_orders_df = complete_df.sortWithinPartitions('customer_id')

optimized_complete_df = sorted_orders_df.join(sorted_customers_df,'customer_id')

### **Bucket Join**

In [13]:
bucketed_customers_df = customers_df.repartition(10, 'customer_id')
bucketed_orders_df = complete_df.repartition(10, 'customer_id')

bucketed_optimized_complete_df = sorted_orders_df.join(sorted_customers_df,'customer_id')

# **Saving and retreiving data efficiently inside and outside dataproc**

In [18]:
# reading from parquet
complete_df = spark.read.parquet('/data/olist/olist-merged')

In [19]:
# saving as parquet in a GCS Bucket
complete_df.write.mode('overwrite').parquet('gs://dataproc-staging-us-central1-1019955474270-diwuvdox/temp_data')

25/05/25 23:50:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [20]:
# saving as a table which can be accessed in hive

complete_df.write.mode('overwrite').saveAsTable('complete_df_table')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
25/05/25 23:52:29 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [ ]:
# saving as csv in our hadoop cluster
complete_df.write.mode('overwrite').option('header', 'true').csv('/data/olist/olist-merged/csv')